In [1]:
# Libraries
import sys
sys.path.append('C:\Proyectos\Loteria\DataBase\Scrapping')
from scrapping import get_data
import pandas as pd
import numpy as np
from collections import Counter
from decimal import Decimal, getcontext
getcontext().prec = 4
np.set_printoptions(precision=4)

In [2]:
# Functions
def count_skips(df, list_numbers):
    counts = {key: 0 for key in list_numbers}
    
    for columns in df:
        counter = 0
        for i in reversed(df[columns]):
            if not i:
                counter += 1
            else:
                counts[columns] = counter
                break
                
    return counts

def max_output(df):
    df = df.T
    max_numbers = df.sort_values(by=df.columns[0], ascending=False)
    return max_numbers

def year_hits(database, df_with_numbers, numbers_quantity):
    db_year = database['Dates'].dt.year
    
    def count_hits(year_number):
        year, number = year_number
        filtered = df_with_numbers.loc[(db_year == year) & (df_with_numbers == number).any(axis=1), :]
        count = filtered.eq(number).sum().sum()
        return pd.Series({'Year': year, 'Number': number, 'Count': int(count)})
    
    year_numbers = [(y, n) for y in range(db_year.min(), db_year.max()+1) for n in numbers_quantity]
    year_history = pd.DataFrame(year_numbers, columns=['Year', 'Number'])
    year_history = year_history.apply(count_hits, axis=1)
    year_history = year_history.pivot_table(index='Year', columns='Number', values='Count', fill_value=0)
    return year_history

def get_hits(database, df_with_numbers, numbers_quantity):
    hits = year_hits(database, df_with_numbers, numbers_quantity).sum().reset_index()
    hits = hits.rename(columns = {'Number': 'Numbers', 'Count': 'Hits'}).set_index('Numbers').T
    hits = hits.iloc[0].rename('Hits').to_frame()
    hits.index.name = None
    return hits.T

def total_average_hits(database, hits, numbers, is_star=False, aprox=False):
    divide = 2 if is_star else 5
    average_hits = hits.apply(lambda hits: hits / len(database) / divide)
    average_hits = average_hits.iloc[0].rename('Average').to_frame()
    average_hits.index.name = None
    if aprox:
        return Decimal(average_hits['Average'].sum()) / Decimal(int(numbers.max())) + Decimal(0.001)
    else:
        return Decimal(average_hits['Average'].sum()) / Decimal(int(numbers.max()))

def minimal_hits(database, hits, numbers, average, is_star=False, aprox=False):
    min_hits = total_average_hits(database, hits, numbers, is_star, aprox)
    return min_hits * Decimal(int(hits.iloc[0, 0])) / Decimal(float(average.iloc[0, 0]))

def average_hits(database, hits_data, numbers, is_star=False):
    divide = 2 if is_star else 5
    hits_by_num = hits_data.apply(lambda hits: hits / len(database) / divide)
    hits_by_num = hits_by_num.iloc[0].rename('Average').to_frame()
    hits_by_num.index.name = None
    averages = []
    
    for num in numbers:
        avg = hits_by_num.loc[num]['Average'].sum()
        averages.append(round(avg, 6))
        
    result = pd.DataFrame({'Numbers/Stars': numbers, 'Average_Hits': averages})
    result = result.T
    result.columns = result.iloc[0].astype(int)
    result = result[1:]
    return result

def natural_rotation(database, hits, numbers, data_average, index_start, index_end, is_star=False, aprox=False):
    average = total_average_hits(database, hits, numbers, is_star, aprox)
    m_hits = minimal_hits(database, hits, numbers, data_average, is_star, aprox)
    rotation = pd.DataFrame({'Hits': hits.iloc[0], 'Average_Numbers': data_average.iloc[0], 'Average': average, 'Hits_Needed': m_hits}, index = range(index_start, index_end + 1))
    rotation['Difference'] = rotation['Hits'] - rotation['Hits_Needed']
    return rotation

def get_rotations(database, hits, numbers, data_average, is_star=False):
    aprox_rotations = []
    exact_rotations = []
    index_ranges = [(1, 25, '_low'), (26, 50, '_high')]
    
    for start, end, suffix in index_ranges:
        # exact rotations
        exact_rotation = natural_rotation(database, hits, numbers, data_average, start, end, is_star=is_star, aprox=False)
        exact_rotations.append(exact_rotation)
        # approx rotations
        aprox_rotation = natural_rotation(database, hits, numbers, data_average, start, end, is_star=is_star, aprox=True)
        aprox_rotations.append(aprox_rotation)
        
    return tuple(aprox_rotations + exact_rotations)

def combination_count(database, numbers, df_with_numbers):
    def low_numbers(numbers):
        return set(range(1,26)).intersection(numbers)
    
    def make_list(list_of_numbers, count_type=None):
        if count_type == 'low_high':
            return [number in low_numbers(numbers) for number in list_of_numbers]
        elif count_type == 'odd_even':
            return [True if number % 2 != 0 else False for number in list_of_numbers]
        
    low_high = []
    odd_even = []
    for draw in range(len(database)):
        list_of_numbers = df_with_numbers.loc[draw, :].tolist()
        count = Counter(make_list(list_of_numbers, count_type='low_high'))
        low_high.append((count.get(True,0), count.get(False,0)))
        count = Counter(make_list(list_of_numbers, count_type='odd_even'))
        odd_even.append((count.get(True,0), count.get(False,0)))
        
    return low_high, odd_even

def combination_df(database, low_high_counts, odd_even_counts):
    COMBINATIONS = [(3,2), (2,3), (1,4), (4,1), (0,5), (5,0)]
    draws = set(range(0,len(database)))
    columns_id = ['3/2', '2/3', '1/4', '4/1', '0/5', '5/0']
    
    low_high = {}
    odd_even = {}
    for i in draws:
        counts_l_h = {}
        counts_o_e = {}
        for combination in COMBINATIONS:
            count_l_h = sum([1 for j in range(i-9,i+1) if combination[0] == low_high_counts[j][0] and combination[1] == low_high_counts[j][1]])
            counts_l_h[combination] = count_l_h
            count_o_e = sum([1 for j in range(i-9,i+1) if combination[0] == odd_even_counts[j][0] and combination[1] == odd_even_counts[j][1]])
            counts_o_e[combination] = count_o_e
        low_high[i] = counts_l_h
        odd_even[i] = counts_o_e
    
    low_high = pd.DataFrame.from_dict(low_high, orient='index')
    odd_even = pd.DataFrame.from_dict(odd_even, orient='index')
    
    for df in [low_high, odd_even]:
        df.columns = columns_id
    
    for df, name in [(low_high, 'L/H'), (odd_even, 'O/E')]:
        df.columns.name = name
    
    for df in [low_high, odd_even]:
        df.index.name = 'Draws'
    
    return low_high, odd_even

In [3]:
# Rules
# def current_date(database, str_column_with_dates):
#     current_year = database.loc[len(database) - 1, str_column_with_dates].year
#     current_month = databse.loc[len(dabatase) - 1, str_column_with_dates].month
#     return (current_year, current_month)

# def numbers_month(current_year, current_month, numbers):
#     audit = []
#     for number in numbers:
#         data = month_history.loc[(current_year, current_month), numbers]
#         audit.append(data)
#     return audit

In [4]:
# Load the data base and obtain the first DataFrame
db = pd.read_parquet('C:\Proyectos\Loteria\DataBase\db.parquet')
winning_numbers = db.iloc[:, 2:7]
winning_stars = db.iloc[:, 7:9]
total_numbers = np.arange(1, 51)
total_stars = np.arange(1, 13)

# Create a template DataFrame with all values set to False
skip_winners_bool = pd.DataFrame(False, columns=[str(i) for i in range(1, 51)], index=range(len(winning_numbers)))

# Fill in the True values
for e in range(1, 6):
    for i in range(1, 51):
        skip_winners_bool[f"{i}"] |= (winning_numbers[f"Nro{e}"] == i)

# Add an extra row, in order to compare the number that did not appear for the first time in the game history
d_0 = pd.DataFrame(columns = [str(i) for i in range(1, 51)], index=[0]).fillna(True)

# Create the final DataFrame
skip_winners = pd.concat([d_0, skip_winners_bool]).reset_index(drop=True)
del d_0
del skip_winners_bool

# Hits of numbers
numbers_hits = get_hits(db, winning_numbers, total_numbers)

# Hits of stars since draw 940, this is because of the change of rules. The star 12 was added in September 24th 2016
stars_filtered = db[(db['Sorteos'] > 940)].iloc[:, [0, 7, 8]]
stars_filtered_hits = get_hits(stars_filtered, stars_filtered, total_stars)

# Year History for numbers and stars
numbers_year_history = year_hits(db, winning_numbers, total_numbers)
stars_year_history = year_hits(db, stars_filtered, total_stars)
ny_mean = pd.DataFrame(numbers_year_history.mean(), columns=['Average']).T.rename(index={'0': 'Average'})
ny_median = pd.DataFrame(numbers_year_history.median(), columns=['Median']).T.rename(index={'0': 'Median'})
numbers_year_history = pd.concat([ny_median, ny_mean, numbers_year_history])

# Average of hits per numbers
numbers_average = average_hits(db, numbers_hits, total_numbers)

# Average of hits per star
stars_average = average_hits(stars_filtered, stars_filtered_hits, total_stars, is_star=True)

# Natural rotation of the numbers, using exact and aproximation values
aprox_rotation_BN_low, aprox_rotation_BN_high, exact_rotation_BN_low, exact_rotation_BN_high = get_rotations(db, numbers_hits, total_numbers, numbers_average, is_star=False)

In [5]:
# It creates the list of draws, numbers and the dictionary to obtain the amount of skips per number if wins and looses.
draws = list(np.arange(1, len(skip_winners)))
numbers = [str(i) for i in range(1, 51)]
dicts = {draw: {key:[] for key in numbers} for draw in draws}

for e in draws:
    df = skip_winners.loc[:e]
    counts = count_skips(df, numbers)
    dicts[e].update(counts)

skip_numbers = pd.DataFrame(dicts).T
del draws
del numbers

In [6]:
# Order the last draw for skips:
last_draw = sorted(dicts[len(skip_numbers)].items(), key=lambda x: x[1])
last_draw = pd.DataFrame({'Numero': [x[0] for x in last_draw], 'Skips': [int(x[1]) for x in last_draw]})
del dicts

# Select the last 12 draws
last_12_draws = np.arange(len(skip_numbers) - 12, len(skip_numbers) + 1)
sk_12 = skip_numbers.loc[last_12_draws]

# This establish the skips of the last 12 draws
skips = np.arange(0, 19)
aus_12 = [sk_12.loc[i - 1, str(column)] for i in last_12_draws[1:13] for column in sk_12 if sk_12.loc[i, str(column)] == 0]
counter_7 = Counter(aus_12[25:60])
counter_12 = Counter(aus_12)
last_7 = [counter_7.get(i, 0) for i in skips]
last_12 = [counter_12.get(i, 0) for i in skips]
skips_7_12 = pd.DataFrame({'7': last_7, '12': last_12})
del skips
del aus_12

In [7]:
groups = [list(range(i, i + 10)) for i in range(1, 51, 10)]
group_names = [tuple(range(i, i + 10)) for i in range(1, 51, 10)]
results = {i: {group_name: sum([1 for num in row if num in group]) for group_name, group in zip(group_names, groups)} for i, row in winning_numbers.iterrows()}
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['{}_to_{}'.format(i, i + 9) for i in range(1, 51, 10)]
sg_10 = results_df.iloc[-10:]
sg_5 = results_df.iloc[-5:]
groups_df = pd.DataFrame({'10_games': (sg_10 > 0).sum(), '5_games': (sg_5 > 0).sum()}).T

In [8]:
years = db['Dates'].dt.year.to_frame()
years = years.rename(columns={'Dates': 'Years'})
months = db['Dates'].dt.month.to_frame()
months = months.rename(columns={'Dates': 'Month'})
year_month = pd.concat([years, months, winning_numbers], axis=1)
series_dates = year_month.melt(id_vars=['Years', 'Month'], var_name='Data', value_vars=['Nro1', 'Nro2', 'Nro3', 'Nro4', 'Nro5'])
df_dates = series_dates.groupby(['Years', 'Month', 'Data'])['value'].value_counts().unstack(fill_value=0)
cols = ['Nro1', 'Nro2', 'Nro3', 'Nro4', 'Nro5']
month_history = df_dates[df_dates.index.get_level_values('Data').isin(cols)].groupby(level=['Years', 'Month']).sum()

In [9]:
low_high, odd_even = combination_count(db, total_numbers, winning_numbers)
low_high_df, odd_even_df = combination_df(db, low_high, odd_even)
low_high = pd.DataFrame(low_high)
odd_even = pd.DataFrame(odd_even)
LHOE = pd.concat([low_high, odd_even], axis=1)
LHOE.columns = ['L', 'H', 'O', 'E']

In [10]:
COMBINATIONS = [(3,2), (2,3), (1,4), (4,1), (0,5), (5,0)]
lh_count_dic = {i: {key: 0 for key in COMBINATIONS} for i in range(1, len(db) - 99)}
oe_count_dic = {i: {key: 0 for key in COMBINATIONS} for i in range(1, len(db) - 99)}

LH = LHOE.iloc[:, 0:2]
last_row = LHOE.shape[0]
for i in range(1,last_row-99):
    LH_slice = LH.iloc[i-1:i+99]
    for combination in COMBINATIONS:
        j = combination[0]
        e = combination[1]
        coincidencias = LH_slice[(LH_slice['L'] == j) & (LH_slice['H'] == e)]
        suma = coincidencias['L'].count()
        # Update the dictionaries lh_count_dic and oe_count_dic with the calculated value
        lh_count_dic[i].update({combination: suma})
        oe_count_dic[i].update({combination: suma})

In [11]:
db

,Dates,Sorteos,Nro1,Nro2,Nro3,Nro4,Nro5,Star_1,Star_2
0,2004-02-13 00:00:00,1,16,29,32,36,41,7,9
1,2004-02-20 00:00:00,2,7,13,39,47,50,2,5
2,2004-02-27 00:00:00,3,14,18,19,31,37,4,5
3,2004-03-05 00:00:00,4,4,7,33,37,39,1,5
4,2004-03-12 00:00:00,5,15,24,28,44,47,4,5
...,...,...,...,...,...,...,...,...,...
1616,2023-03-21 00:00:00,1617,1,9,20,29,32,2,3
1617,2023-03-24 00:00:00,1618,5,12,25,36,46,6,10
1618,2023-03-28 01:00:00,1619,16,21,34,36,44,9,10
1619,2023-03-31 01:00:00,1620,16,18,28,34,47,5,10


In [12]:
last_draw

,Numero,Skips
0,10,0
1,16,0
2,31,0
3,33,0
4,50,0
5,18,1
6,28,1
7,34,1
8,47,1
9,21,2


In [13]:
skips_7_12

,7,12
0,6,9
1,2,4
2,2,3
3,1,3
4,1,3
5,2,3
6,2,5
7,3,5
8,2,2
9,2,2


In [14]:
results_df

,1_to_10,11_to_20,21_to_30,31_to_40,41_to_50
0,0,1,1,2,1
1,1,1,0,1,2
2,0,3,0,2,0
3,2,0,0,3,0
4,0,1,2,0,2
...,...,...,...,...,...
1616,2,1,1,1,0
1617,1,1,1,1,1
1618,0,1,1,2,1
1619,0,2,1,1,1


In [15]:
groups_df

,1_to_10,11_to_20,21_to_30,31_to_40,41_to_50
10_games,6,9,8,9,9
5_games,3,5,4,5,4


In [16]:
numbers_hits

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
Hits,163,150,163,163,171,159,161,146,153,173,...,138,183,156,179,166,138,146,154,161,181


In [17]:
numbers_year_history

Number,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
Median,8.50,7.5,8.00,8.00,8.00,8.00,8.50,7.0,7.50,8.50,...,6.5,9.50,8.0,9.50,10.0,7.0,7.0,7.5,7.50,9.00
Average,8.15,7.5,8.15,8.15,8.55,7.95,8.05,7.3,7.65,8.65,...,6.9,9.15,7.8,8.95,8.3,6.9,7.3,7.7,8.05,9.05
2004,10.00,3.0,5.00,8.00,4.00,5.00,5.00,4.0,4.00,8.00,...,5.0,4.00,5.0,6.00,3.0,1.0,4.0,3.0,5.00,4.00
2005,6.00,4.0,9.00,4.00,1.00,8.00,5.00,7.0,3.00,4.00,...,5.0,8.00,6.0,5.00,3.0,2.0,9.0,5.0,3.00,10.00
2006,8.00,4.0,8.00,5.00,8.00,6.00,4.00,8.0,9.00,7.00,...,4.0,1.00,5.0,7.00,6.0,2.0,6.0,3.0,8.00,12.00
2007,4.00,6.0,6.00,4.00,6.00,3.00,7.00,4.0,5.00,3.00,...,9.0,6.00,5.0,5.00,7.0,3.0,1.0,7.0,5.00,3.00
2008,3.00,7.0,3.00,6.00,5.00,4.00,9.00,6.0,8.00,5.00,...,4.0,3.00,0.0,5.00,10.0,4.0,3.0,3.0,6.00,14.00
2009,0.00,3.0,1.00,8.00,8.00,7.00,4.00,5.0,7.00,2.00,...,4.0,7.00,5.0,5.00,2.0,7.0,7.0,3.0,5.00,4.00
2010,7.00,5.0,5.00,10.00,4.00,4.00,5.00,4.0,9.00,2.00,...,3.0,7.00,8.0,4.00,5.0,11.0,4.0,2.0,7.00,4.00
2011,9.00,7.0,5.00,9.00,7.00,8.00,5.00,4.0,6.00,6.00,...,8.0,8.00,3.0,9.00,12.0,7.0,8.0,11.0,8.00,14.00


In [18]:
aprox_rotation_BN_low

,Hits,Average_Numbers,Average,Hits_Needed,Difference
1,163,0.020111,0.02100,170.2,-7.2
2,150,0.018507,0.02100,170.2,-20.2
3,163,0.020111,0.02100,170.2,-7.2
4,163,0.020111,0.02100,170.2,-7.2
5,171,0.021098,0.02100,170.2,0.8
6,159,0.019618,0.02100,170.2,-11.2
7,161,0.019864,0.02100,170.2,-9.2
8,146,0.018014,0.02100,170.2,-24.2
9,153,0.018877,0.02100,170.2,-17.2
10,173,0.021345,0.02100,170.2,2.8


In [19]:
aprox_rotation_BN_high

,Hits,Average_Numbers,Average,Hits_Needed,Difference
26,178,0.021962,0.02100,170.2,7.8
27,172,0.021221,0.02100,170.2,1.8
28,159,0.019618,0.02100,170.2,-11.2
29,173,0.021345,0.02100,170.2,2.8
30,165,0.020358,0.02100,170.2,-5.2
31,156,0.019247,0.02100,170.2,-14.2
32,151,0.018630,0.02100,170.2,-19.2
33,136,0.016780,0.02100,170.2,-34.2
34,157,0.019371,0.02100,170.2,-13.2
35,161,0.019864,0.02100,170.2,-9.2


In [20]:
exact_rotation_BN_low

,Hits,Average_Numbers,Average,Hits_Needed,Difference
1,163,0.020111,0.02,162.1,0.9
2,150,0.018507,0.02,162.1,-12.1
3,163,0.020111,0.02,162.1,0.9
4,163,0.020111,0.02,162.1,0.9
5,171,0.021098,0.02,162.1,8.9
6,159,0.019618,0.02,162.1,-3.1
7,161,0.019864,0.02,162.1,-1.1
8,146,0.018014,0.02,162.1,-16.1
9,153,0.018877,0.02,162.1,-9.1
10,173,0.021345,0.02,162.1,10.9


In [21]:
exact_rotation_BN_high

,Hits,Average_Numbers,Average,Hits_Needed,Difference
26,178,0.021962,0.02,162.1,15.9
27,172,0.021221,0.02,162.1,9.9
28,159,0.019618,0.02,162.1,-3.1
29,173,0.021345,0.02,162.1,10.9
30,165,0.020358,0.02,162.1,2.9
31,156,0.019247,0.02,162.1,-6.1
32,151,0.018630,0.02,162.1,-11.1
33,136,0.016780,0.02,162.1,-26.1
34,157,0.019371,0.02,162.1,-5.1
35,161,0.019864,0.02,162.1,-1.1


In [22]:
stars_filtered_hits

,1,2,3,4,5,6,7,8,9,10,11,12
Hits,99,138,143,105,92,124,106,115,105,101,121,113


In [23]:
stars_year_history

Number,1,2,3,4,5,6,7,8,9,10,11,12
Year,,,,,,,,,,,,
2004,0,0,0,0,0,0,0,0,0,0,0,0
2005,0,0,0,0,0,0,0,0,0,0,0,0
2006,0,0,0,0,0,0,0,0,0,0,0,0
2007,0,0,0,0,0,0,0,0,0,0,0,0
2008,0,0,0,0,0,0,0,0,0,0,0,0
2009,0,0,0,0,0,0,0,0,0,0,0,0
2010,0,0,0,0,0,0,0,0,0,0,0,0
2011,0,0,0,0,0,0,0,0,0,0,0,0
2012,0,0,0,0,0,0,0,0,0,0,0,0
